<a href="https://colab.research.google.com/github/NickJordan289/CryptoExchangeHelper/blob/master/Trading_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install python-dotenv

In [0]:
import os
import requests
from pprint import pprint
from dotenv import load_dotenv, find_dotenv

In [3]:
#@title Setup { run: "auto", vertical-output: true }

print(find_dotenv())
load_dotenv(find_dotenv(), verbose=True)

token = os.getenv("TOKEN")
head = {
    'Authorization': f'Token token="{token}"'
}
investment = 400 #@param {type:"number"}
base_currency = 'AUD' #@param {type:"string"}

/content/.env
_g_ibjyKJ-d4mcJuWjNq4l9VQQ22iYOOxI1YtABLRf0


In [0]:
# GET
orders_url = 'https://api.exchange.coinjar.com/orders'

req = requests.get(orders_url, headers=head)
orders = req.json()
#pprint(orders)

In [5]:
# GET
def getLastPrice(id):
  ticker_url = f'https://api.exchange.coinjar.com/products/{id}{base_currency}/ticker'

  req = requests.get(ticker_url, headers=head)
  ticker = req.json()
  return float(ticker['last'])

last = getLastPrice('XRP')
print(f'{id} Last Price : {last}')

<built-in function id> Last Price : 0.3565


In [0]:
#Solves for xrp exchange price using known variables
from sympy.solvers import solve
from sympy import Symbol
x = Symbol('x')
def solve_xrp(bank,investment,xrp, profit=0):
  return solve((x*xrp)+bank-(investment*(1+profit)))[0]

In [7]:
#@title Calculate Profit { run: "auto", vertical-output: true }
accounts_url = 'https://api.exchange.coinjar.com/accounts'

req = requests.get(accounts_url, headers=head)
balances = req.json()
#pprint(balances)

print(f"{'-'*10} Accounts {'-'*10}")
bals = {}
for bal in balances:
  if(float(bal['settled_balance']) > 0):
    if(bal['asset_code'] == base_currency):
      bank = float(bal['settled_balance'])
    else:
      bals[bal['asset_code']] = float(bal['settled_balance'])
    print(f"{bal['asset_code']} : {bal['settled_balance']}")
print()

prices = {}
evaluation = bank
for i,k in enumerate(bals):
  price = getLastPrice(id=k)
  prices[k] = price
  evaluation = evaluation + (bals[k]*price)

print(f"{'-'*5} Account Evaluation {'-'*5}")
print(f'Evaluation : ${evaluation} {base_currency}')
print(f'Profit if sold at current price : ${round(evaluation-investment,2)} {base_currency}')
print()

print(f"{'-'*2} Calculated Recommendations {'-'*2}")
min_val = solve_xrp(xrp=bals['XRP'], bank=bank, investment=investment)
print(f'min to break even {round(min_val,4)}')
desired_profit_percent = 5 #@param {type:"number"}
desired_profit_decimal = desired_profit_percent/100.0
profitable = solve_xrp(xrp=bals['XRP'], bank=bank, investment=investment, profit=desired_profit_decimal)
print(f"profiting at {desired_profit_percent}% : {round(profitable,4)} : +${round(profitable*bals['XRP']+bank,2)-investment} {base_currency}")

---------- Accounts ----------
AUD : 35.36
XRP : 1000.000000

----- Account Evaluation -----
Evaluation : $391.86 AUD
Profit if sold at current price : $-8.14 AUD

-- Calculated Recommendations --
min to break even 0.3646
profiting at 5% : 0.3846 : +$20.0 AUD


In [8]:
total = 0
for order in orders:
  total = total + (float(order['size']) * float((order['price'])))
evaluation =  round(total+bank, 2)
profit = round(evaluation-investment, 2)
print(f'Total assuming orders are filled ${evaluation} {base_currency}')
print(f'Profit assuming orders are filled ${profit} {base_currency}')

Total assuming orders are filled $421.2 AUD
Profit assuming orders are filled $21.2 AUD


In [0]:
## POST
#create_url = 'https://api.exchange.coinjar.com/orders'
#
#data = {
#    'product_id' : 'BTCAUD',
#    'type' : 'LMT',
#    'side' : 'buy',
#    'price' : '12000.00',
#    'size' : '0.008',
#    'time_in_force' : 'GTC'
#}
#
#req = requests.post(create_url, headers=head, json=data)
#print(req.json())